In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import SQLContext
from pyspark import SparkConf
from pyspark.sql.window import Window
import pyodbc

conf = SparkConf() \
    .set("spark.jars.packages",
        "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2,org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .setMaster("local[*]") \
    .setAppName("ReadMongoDB")


sc = SparkContext(conf=conf)
sqlC = SQLContext(sc)

mongo_ip = "mongodb://localhost:27017/rugs."
rug = sqlC.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", mongo_ip +"rugs_collection").load()


c:\Users\tram2\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\sql\context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
rug.show()

+----------------+--------------------+--------------------+---------+-------+--------------------+-----------+--------------------+-------+-------------+--------------------+------+--------------------+
|Cleaning_Process|            Material|                Name|Old_Price| Origin|             Pattern|Pile_height|     Product_Reviews|   Save|Special_Price|                Type|Weight|                 _id|
+----------------+--------------------+--------------------+---------+-------+--------------------+-----------+--------------------+-------+-------------+--------------------+------+--------------------+
|   Easy to clean|           Polyester|Cleo 013-0016 619...|    223.2|Belgium|            Abstract|       25.0|                 N/A|Save 24|       169.08|              Modern|  2900|{6708ecdbc59a34b2...|
|   Easy to clean|       Polypropylene|Nova NV10 Antique...|     59.0| Turkey|              Floral|        9.0|Excellent distres...|Save 33|         39.6|         Traditional|  2250|{6

In [3]:


kafka_bootstrap_servers = "localhost:9092"
topic_rugs = "rug"

def write_to_kafka(df, topic):
    # Hiển thị trước 5 dòng dữ liệu để kiểm tra
    df.show(5)
    
    # Các cột cần xử lý NaN
    columns_to_fill = ['Cleaning_Process', 'Material', 'Name', 'Old_Price', 'Origin', 
                    'Pattern', 'Pile_height', 'Product_Reviews', 'Save', 
                    'Special_Price', 'Type', 'Weight']
    
    # Giá trị mặc định cho các cột
    fill_values = {
        'Cleaning_Process': 'Unknown',
        'Material': 'Unknown',
        'Name': 'Unnamed',
        'Old_Price': 0.0,
        'Origin': 'Unknown',
        'Pattern': 'Unknown',
        'Pile_height': 0.0,
        'Product_Reviews': 'No reviews',
        'Save': 'No discount',
        'Special_Price': 0.0,
        'Type': 'Uncategorized',
        'Weight': 0.0
    }
    
    # Điền giá trị mặc định cho các cột thiếu
    df = df.fillna({col: fill_values.get(col, None) for col in columns_to_fill if col in df.columns})
    
    # Hiển thị dữ liệu đã được xử lý NaN
    df.show(5)
    
    # In schema của dataframe
    df.printSchema()
    
    # Thêm cột `_id` nếu chưa có và chuyển thành kiểu chuỗi
    if "_id" not in df.columns:
        df = df.withColumn("_id", F.monotonically_increasing_id())
    df = df.withColumn("_id", F.col("_id").cast("string"))
    
    # Chuyển đổi các cột thành định dạng Kafka key-value
    df = df.selectExpr("CAST(_id AS STRING) AS key", "to_json(struct(*)) AS value")
    
    # Ghi dữ liệu vào Kafka
    df.write \
        .format("kafka") \
        .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
        .option("topic", topic) \
        .option("checkpointLocation", "/tmp/kafka_checkpoint") \
        .save()

# Gọi hàm với dataframe và topic
write_to_kafka(rug, topic_rugs)


+----------------+--------------------+--------------------+---------+-------+--------+-----------+--------------------+-------+-------------+--------------------+------+--------------------+
|Cleaning_Process|            Material|                Name|Old_Price| Origin| Pattern|Pile_height|     Product_Reviews|   Save|Special_Price|                Type|Weight|                 _id|
+----------------+--------------------+--------------------+---------+-------+--------+-----------+--------------------+-------+-------------+--------------------+------+--------------------+
|   Easy to clean|           Polyester|Cleo 013-0016 619...|    223.2|Belgium|Abstract|       25.0|                 N/A|Save 24|       169.08|              Modern|  2900|{6708ecdbc59a34b2...|
|   Easy to clean|       Polypropylene|Nova NV10 Antique...|     59.0| Turkey|  Floral|        9.0|Excellent distres...|Save 33|         39.6|         Traditional|  2250|{6708ece0c59a34b2...|
|   Easy to clean|Polyester , Polyp...|A